In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(rc={'figure.figsize':(10, 7)}, style='darkgrid')
sns.set_color_codes()

from scipy.stats import norm

import cv2

import os

import glob
import datetime
import itertools
import random
from glob import glob
%matplotlib inline


from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the CSV dataset
data = pd.read_csv('/kaggle/input/nih-chest-x-ray-models/87k_diseases_mc.csv')
data = data.drop('No Finding', axis=1)
value_to_delete = 'No Finding'

#Delete rows where column 'A' has the specified value
data = data[data['Finding Labels'] != value_to_delete]

# Split the dataset into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Print the shapes of the train and test datasets
print("Train dataset shape:", train_data.shape)
print("Test dataset shape:", test_data.shape)

data = pd.read_csv('../input/data/Data_Entry_2017.csv')
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('..', 'input','data', 'images*', '*', '*.png'))}
print('Scans found:', len(all_image_paths), ', Total Headers', data.shape[0])
data['path'] = data['Image Index'].map(all_image_paths.get)
data['Patient Age'] = data['Patient Age'].map(lambda x: int(x[:-1])if isinstance(x, str)else x)
data.sample(3)


In [ ]:
label_counts = data['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
data

In [ ]:
data['Finding Labels'] = data['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
from itertools import chain
all_labels = np.unique(list(chain(*data['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        data[c_label] = data['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
data.sample(3)

In [ ]:
# keep at least 1000 cases
MIN_CASES = 1000
all_labels = [c_label for c_label in all_labels if data[c_label].sum()>MIN_CASES]
print('Clean Labels ({})'.format(len(all_labels)), 
      [(c_label,int(data[c_label].sum())) for c_label in all_labels])

In [ ]:
data

In [ ]:
# since the dataset is very unbiased, we can resample it to be a more reasonable collection
# weight is 0.1 + number of findings
sample_weights = data['Finding Labels'].map(lambda x: len(x.split('|')) if len(x)>0 else 0).values + 4e-2
sample_weights /= sample_weights.sum()
data = data.sample(100000, weights=sample_weights)

In [ ]:
label_counts = data['Finding Labels'].value_counts()[:15]
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
_ = ax1.set_xticklabels(label_counts.index, rotation = 90)

In [ ]:
label_counts = 100*np.mean(data[all_labels].values,0)
fig, ax1 = plt.subplots(1,1,figsize = (12, 8))
ax1.bar(np.arange(len(label_counts))+0.5, label_counts)
ax1.set_xticks(np.arange(len(label_counts))+0.5)
ax1.set_xticklabels(all_labels, rotation = 90)
ax1.set_title('Adjusted Frequency of Diseases in Patient Group')
_ = ax1.set_ylabel('Frequency (%)')

In [ ]:
data = data.drop('Follow-up #', axis=1)
data = data.drop('Patient ID', axis=1)
data = data.drop('Patient Age', axis=1)
data = data.drop('Patient Gender', axis=1)
data = data.drop('View Position', axis=1)
data = data.drop('OriginalImage[Width', axis=1)
data = data.drop('Height]', axis=1)
data = data.drop('OriginalImagePixelSpacing[x', axis=1)
data = data.drop('y]', axis=1)
data = data.drop('Unnamed: 11', axis=1)

In [ ]:
data

In [ ]:
data['Finding Labels'].value_counts()

In [ ]:
data['disease_vec'] = data.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])

In [ ]:
counts = data['Finding Labels'].value_counts()
mask = data['Finding Labels'].isin(counts[counts >= 251].index)
data = data[mask]

In [ ]:

# Threshold and category to delete
threshold = 0.75
category_to_delete = ''

# Select rows with the specified category
category_rows = data[data['Finding Labels'] == category_to_delete]

# Calculate the number of rows to delete
num_rows_to_delete = int(len(category_rows) * threshold)

# Randomly select rows to delete
rows_to_delete = np.random.choice(category_rows.index, size=num_rows_to_delete, replace=False)

# Delete selected rows from the DataFrame
data = data.drop(rows_to_delete)



In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(data, 
                                   test_size = 0.25, 
                                   random_state = 2018,
                                   stratify = data['Finding Labels'].map(lambda x: x[:4]))
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (128, 128)
core_idg = ImageDataGenerator(samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.05, 
                              width_shift_range=0.1, 
                              rotation_range=5, 
                              shear_range = 0.1,
                              fill_mode = 'reflect',
                              zoom_range=0.15)

In [ ]:
def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen

In [ ]:
valid_df['newLabel'] = valid_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)
train_df['newLabel'] = train_df.apply(lambda x: x['Finding Labels'].split('|'), axis=1)


In [ ]:
train_df

In [ ]:
train_gen = core_idg.flow_from_dataframe(dataframe=train_df,
directory=None,
x_col = 'path',
y_col = 'newLabel',
class_mode = 'categorical',
classes = all_labels,
target_size = IMG_SIZE,
color_mode = 'rgb',
batch_size = 32)

valid_gen = core_idg.flow_from_dataframe(dataframe=valid_df,
directory=None,
x_col = 'path',
y_col = 'newLabel',
class_mode = 'categorical',
classes = all_labels,
target_size = IMG_SIZE,
color_mode = 'rgb',
batch_size = 256) # we can use much larger batches for evaluation

test_X, test_Y = next(core_idg.flow_from_dataframe(dataframe=valid_df,
directory=None,
x_col = 'path',
y_col = 'newLabel',
class_mode = 'categorical',
classes = all_labels,
target_size = IMG_SIZE,
color_mode = 'rgb',
batch_size = 1024))

In [ ]:
t_x, t_y = next(train_gen)
fig, m_axs = plt.subplots(4, 4, figsize = (16, 16))
for (c_x, c_y, c_ax) in zip(t_x, t_y, m_axs.flatten()):
    c_ax.imshow(c_x[:,:,0], cmap = 'bone', vmin = -1.5, vmax = 1.5)
    c_ax.set_title(', '.join([n_class for n_class, n_score in zip(all_labels, c_y) 
                             if n_score>0.5]))
    c_ax.axis('off')

In [ ]:
len(train_df)

In [ ]:
EPOCHS = 20

# DenseNet121

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

base_model = DenseNet121(input_shape =  t_x.shape[1:], 
                                 include_top = False, weights = 'imagenet')

optimizer = Adam(learning_rate=0.001)

multi_disease_model = Sequential()
multi_disease_model.add(base_model)
multi_disease_model.add(GlobalAveragePooling2D())
#multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(512))
#multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(len(all_labels), activation = 'sigmoid'))
multi_disease_model.compile(optimizer = optimizer, loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy', 'mae'])
multi_disease_model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('xray_class')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5)
callbacks_list = [checkpoint, early]

In [ ]:
history = multi_disease_model.fit_generator(train_gen, 
                                  steps_per_epoch=100,
                                  validation_data = (test_X, test_Y), 
                                  epochs = 20, 
                                  callbacks = callbacks_list)

In [ ]:
for c_label, s_count in zip(all_labels, 100*np.mean(test_Y,0)):
    print('%s: %2.2f%%' % (c_label, s_count))

In [ ]:
pred_Y = multi_disease_model.predict(test_X, batch_size = 32, verbose = True)

In [ ]:
from sklearn.metrics import roc_curve, auc
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for (idx, c_label) in enumerate(all_labels):
    fpr, tpr, thresholds = roc_curve(test_Y[:,idx].astype(int), pred_Y[:,idx])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('barely_trained_net.png')

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history.history['binary_accuracy'], linestyle='-')

plt.title('DenseNet121 - Train - Accuracy', fontweight='bold', fontsize=16)
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history.history['loss'], linestyle='-')

plt.title('DenseNet121 - Loss ', fontweight='bold', fontsize=16)
plt.ylabel('loss')
plt.xlabel('epoch')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history.history['val_binary_accuracy'], linestyle='-')

plt.title('DenseNet121 - Validation Accuracy', fontweight='bold', fontsize=16)
plt.ylabel('val_acc')
plt.xlabel('epoch')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history.history['val_loss'], linestyle='-')

plt.title('DenseNet121 - Validation Loss ', fontweight='bold', fontsize=16)
plt.ylabel('val_loss')
plt.xlabel('epoch')

plt.show()

In [ ]:
# Save the model
multi_disease_model.save('ds121_sigmoid_v2.h5')

### MobileNetV2

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


# 1. Create model
base_mobilenet_model = MobileNetV2(input_shape =  t_x.shape[1:], 
                                 include_top = False, weights = 'imagenet')
optimizer = Adam(learning_rate=0.001)
multi_disease_model = Sequential()
multi_disease_model.add(base_mobilenet_model)
multi_disease_model.add(GlobalAveragePooling2D())
#multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(512))
#multi_disease_model.add(Dropout(0.5))
multi_disease_model.add(Dense(len(all_labels), activation = 'sigmoid'))
multi_disease_model.compile(optimizer = optimizer, loss = 'binary_crossentropy',
                           metrics = ['binary_accuracy', 'mae'])
multi_disease_model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('xray_class')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=5)
callbacks_list = [checkpoint, early]

In [ ]:
history = multi_disease_model.fit_generator(train_gen, 
                                  steps_per_epoch=100,
                                  validation_data = (test_X, test_Y), 
                                  epochs = EPOCHS, 
                                  callbacks = callbacks_list)

In [ ]:
for c_label, s_count in zip(all_labels, 100*np.mean(test_Y,0)):
    print('%s: %2.2f%%' % (c_label, s_count))

In [ ]:
pred_Y = multi_disease_model.predict(test_X, batch_size = 32, verbose = True)

In [ ]:
from sklearn.metrics import roc_curve, auc
fig, c_ax = plt.subplots(1,1, figsize = (9, 9))
for (idx, c_label) in enumerate(all_labels):
    fpr, tpr, thresholds = roc_curve(test_Y[:,idx].astype(int), pred_Y[:,idx])
    c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
fig.savefig('barely_trained_net.png')

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history.history['binary_accuracy'], linestyle='-')

plt.title('MobileNetV2 - Train - Accuracy - 1K sharpening', fontweight='bold', fontsize=16)
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history.history['loss'], linestyle='-')

plt.title('MobileNetV2 - Loss - 1K sharpening', fontweight='bold', fontsize=16)
plt.ylabel('loss')
plt.xlabel('epoch')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history.history['val_binary_accuracy'], linestyle='-')

plt.title('MobileNetV2 - Validation Accuracy - 1K sharpening', fontweight='bold', fontsize=16)
plt.ylabel('val_acc')
plt.xlabel('epoch')

plt.show()

In [ ]:
plt.figure(figsize=(15,5))

plt.plot(history.history['val_loss'], linestyle='-')

plt.title('MobileNetV2 - Validation Loss - 1K sharpening', fontweight='bold', fontsize=16)
plt.ylabel('val_loss')
plt.xlabel('epoch')

plt.show()

In [ ]:
# Save the model
multi_disease_model.save('mobile_sigmoid.h5')

### ResNet50V2

## Load and predict for sharpening

In [ ]:
test_datagen_aug = ImageDataGenerator(rescale=1/255., preprocessing_function=sharpening)

IMG_SHAPE = (224, 224)
BATCH_SIZE = 32

print("Testing images:")
test_data_aug = test_datagen_aug.flow_from_dataframe(test_data,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=42,
                                             batch_size=BATCH_SIZE)

In [ ]:
# Save predictions
def save_to_npy(filename, predictions):
    with open(filename, 'wb') as f:
        np.save(f, predictions)

# Load predictions
def load_saved_preds(filename):
    with open(filename, 'rb') as f:
        a = np.load(f)
    return a

In [ ]:
def load_and_predict(test_data_aug, path, save_file_name):
    model = tf.keras.models.load_model(path)
    preds = model.predict(test_data_aug, steps=len(test_data_aug), verbose=1)
    save_to_npy(save_file_name, preds)

In [ ]:
# load_and_predict(test_data_aug, "../input/nih-chest-x-ray-models/vgg19_1K.h5", "vgg19_1K_npy.npy")
# load_and_predict(test_data_aug, "../input/nih-chest-x-ray-models/ds121_1K.h5", "ds121_1K_npy.npy")
# load_and_predict(test_data_aug, "../input/nih-chest-x-ray-models/nasnet_1K.h5", "nasnet_1K_npy.npy")
# load_and_predict(test_data_aug, "../input/nih-chest-x-ray-models/mobile_1K.h5", "mobile_1K_npy.npy")

load_and_predict(test_data_aug, "vgg19_1K.h5", "vgg19_1K_npy.npy")
load_and_predict(test_data_aug, "ds121_1K.h5", "ds121_1K_npy.npy")
load_and_predict(test_data_aug, "nasnet_1K.h5", "nasnet_1K_npy.npy")
load_and_predict(test_data_aug, "mobile_1K.h5", "mobile_1K_npy.npy")

In [ ]:
test_datagen_aug = ImageDataGenerator(rescale=1/255., preprocessing_function=sharpening)

IMG_SHAPE = (299, 299)
BATCH_SIZE = 32

print("Testing images:")
test_data_aug = test_datagen_aug.flow_from_dataframe(test_data,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=42,
                                             batch_size=BATCH_SIZE)

In [ ]:
# load_and_predict(test_data_aug, "../input/nih-chest-x-ray-models/iv3_1K.h5", "iv3_1K_npy.npy")
load_and_predict(test_data_aug, "iv3_1K.h5", "iv3_1K_npy.npy")

In [ ]:
test_datagen_aug = ImageDataGenerator(preprocessing_function=sharpening)

IMG_SHAPE = (224, 224)
BATCH_SIZE = 32

print("Testing images:")
test_data_aug = test_datagen_aug.flow_from_dataframe(test_data,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=42,
                                             batch_size=BATCH_SIZE)

In [ ]:
# load_and_predict(test_data_aug, "../input/nih-chest-x-ray-models/effb0_1K.h5", "effb0_1K_npy.npy")
# load_and_predict(test_data_aug, "../input/nih-chest-x-ray-models/resnet50_1K.h5", "resnet50_1K_npy.npy")

load_and_predict(test_data_aug, "effb0_1K.h5", "effb0_1K_npy.npy")
load_and_predict(test_data_aug, "resnet50_1K.h5", "resnet50_1K_npy.npy")

### Load predictions

#### Show accuracy, precision, recall, f1

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
BATCH_SIZE = 32
IMG_SHAPE = (224, 224)

test_datagen_aug = ImageDataGenerator(rescale=1/255., preprocessing_function=sharpening)

print("Testing images:")
test_data_aug = test_datagen_aug.flow_from_dataframe(test_data,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=42,
                                             batch_size=BATCH_SIZE)

In [ ]:
vgg19_pred_sharpening = load_saved_preds("vgg19_1K_npy.npy")
ds_121_pred_sharpening = load_saved_preds("ds121_1K_npy.npy")
nasnet_pred_sharpening = load_saved_preds("nasnet_1K_npy.npy")
mobile_v2_pred_sharpening = load_saved_preds("mobile_1K_npy.npy")
effb0_pred_sharpening = load_saved_preds("effb0_1K_npy.npy")
res50v2_pred_sharpening = load_saved_preds("resnet50_1K_npy.npy")
inceptionv3_pred_sharpening = load_saved_preds("iv3_1K_npy.npy")

# vgg19_pred_sharpening = load_saved_preds("vgg19_1K_npy.npy")
# ds_121_pred_sharpening = load_saved_preds("ds121_1K_npy.npy")
# nasnet_pred_sharpening = load_saved_preds("nasnet_1K_npy.npy")
# mobile_v2_pred_sharpening = load_saved_preds("mobile_1K_npy.npy")
# effb0_pred_sharpening = load_saved_preds("effb0_1K_npy.npy")
# res50v2_pred_sharpening = load_saved_preds("resnet50_1K_npy.npy")
# inceptionv3_pred_sharpening = load_saved_preds("iv3_1K_npy.npy")

##### InceptionV3

In [ ]:
iv3_cr = classification_report(test_data_aug.labels.argmax(axis=1), inceptionv3_pred_sharpening.argmax(axis=1), output_dict=True)

iv3_cr = pd.DataFrame(iv3_cr)
iv3_cr.columns = [*labels, "accuracy", "macro avg", "weighted avg"]
iv3_cr

In [ ]:
iv3_cr.drop(["accuracy", "macro avg", "weighted avg"], axis=1, inplace=True)

In [ ]:
iv3_cr = iv3_cr.T

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(iv3_cr['precision'] * 100), x=iv3_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("InceptionV3 - Precision Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(iv3_cr['recall'] * 100), x=iv3_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("InceptionV3 - Recall Score (sharpening) (1K)", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(iv3_cr['f1-score'] * 100), x=iv3_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("InceptionV3 - F1 Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

##### ResNet50V2

In [ ]:
r50_cr = classification_report(test_data_aug.labels.argmax(axis=1), res50v2_pred_sharpening.argmax(axis=1), output_dict=True)

r50_cr = pd.DataFrame(r50_cr)
r50_cr.columns = [*labels, "accuracy", "macro avg", "weighted avg"]
r50_cr

In [ ]:
r50_cr.drop(["accuracy", "macro avg", "weighted avg"], axis=1, inplace=True)

In [ ]:
r50_cr = r50_cr.T

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(r50_cr['precision'] * 100), x=r50_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("ResNet50V2 - Precision Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(r50_cr['recall'] * 100), x=r50_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("ResNet50V2 - Recall Score (sharpening) (1K)", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(r50_cr['f1-score'] * 100), x=r50_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("ResNet50V2 - F1 Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

##### EfficientNetB0

In [ ]:
effb0_cr = classification_report(test_data_aug.labels.argmax(axis=1), effb0_pred_sharpening.argmax(axis=1), output_dict=True)

effb0_cr = pd.DataFrame(effb0_cr)
effb0_cr.columns = [*labels, "accuracy", "macro avg", "weighted avg"]
effb0_cr

In [ ]:
effb0_cr.drop(["accuracy", "macro avg", "weighted avg"], axis=1, inplace=True)

In [ ]:
effb0_cr = effb0_cr.T

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(effb0_cr['precision'] * 100), x=effb0_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("EfficientNetB0 - Precision Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(effb0_cr['recall'] * 100), x=effb0_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("EfficientNetB0 - Recall Score (sharpening) (1K)", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(effb0_cr['f1-score'] * 100), x=effb0_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("EfficientNetB0 - F1 Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

##### MobileNetV2

In [ ]:
mb_cr = classification_report(test_data_aug.labels.argmax(axis=1), mobile_v2_pred_sharpening.argmax(axis=1), output_dict=True)

mb_cr = pd.DataFrame(mb_cr)
mb_cr.columns = [*labels, "accuracy", "macro avg", "weighted avg"]
mb_cr

In [ ]:
mb_cr.drop(["accuracy", "macro avg", "weighted avg"], axis=1, inplace=True)

In [ ]:
mb_cr = mb_cr.T

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(mb_cr['precision'] * 100), x=mb_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("MobileNetV2 - Precision Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(mb_cr['recall'] * 100), x=mb_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("MobileNetV2 - Recall Score (sharpening) (1K)", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(mb_cr['f1-score'] * 100), x=mb_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("MobileNetV2 - F1 Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

##### NasNet Mobile

In [ ]:
nas_cr = classification_report(test_data_aug.labels.argmax(axis=1), nasnet_pred_sharpening.argmax(axis=1), output_dict=True)

nas_cr = pd.DataFrame(nas_cr)
nas_cr.columns = [*labels, "accuracy", "macro avg", "weighted avg"]
nas_cr

In [ ]:
nas_cr.drop(["accuracy", "macro avg", "weighted avg"], axis=1, inplace=True)

In [ ]:
nas_cr = nas_cr.T

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(nas_cr['precision'] * 100), x=nas_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("NasNet - Precision Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(nas_cr['recall'] * 100), x=nas_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("NasNet - Recall Score (sharpening) (1K)", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(nas_cr['f1-score'] * 100), x=nas_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("NasNet - F1 Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

##### VGG19

In [ ]:
vgg19_cr = classification_report(test_data_aug.labels.argmax(axis=1), vgg19_pred_sharpening.argmax(axis=1), output_dict=True)

vgg19_cr = pd.DataFrame(vgg19_cr)
vgg19_cr.columns = [*labels, "accuracy", "macro avg", "weighted avg"]
vgg19_cr

In [ ]:
vgg19_cr.drop(["accuracy", "macro avg", "weighted avg"], axis=1, inplace=True)

In [ ]:
vgg19_cr = vgg19_cr.T

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(vgg19_cr['precision'] * 100), x=vgg19_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("VGG19 - Precision Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(vgg19_cr['recall'] * 100), x=vgg19_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("VGG19 - Recall Score (sharpening) (1K)", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(vgg19_cr['f1-score'] * 100), x=vgg19_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("VGG19 - F1 Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

##### DenseNet121

In [ ]:
ds_121_cr = classification_report(test_data_aug.labels.argmax(axis=1), ds_121_pred_sharpening.argmax(axis=1), output_dict=True)

ds_121_cr = pd.DataFrame(ds_121_cr)
ds_121_cr.columns = [*labels, "accuracy", "macro avg", "weighted avg"]
ds_121_cr

In [ ]:
ds_121_cr.drop(["accuracy", "macro avg", "weighted avg"], axis=1, inplace=True)

In [ ]:
ds_121_cr = ds_121_cr.T

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(ds_121_cr['precision'] * 100), x=ds_121_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("DenseNet121 - Precision Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(ds_121_cr['recall'] * 100), x=ds_121_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("DenseNet121 - Recall Score (sharpening) (1K)", fontsize=16, fontweight="bold");

In [ ]:
plt.figure(figsize=(15, 10))
ax = sns.barplot(y=round(ds_121_cr['f1-score'] * 100), x=ds_121_cr.index, color="brown");
for container in ax.containers:
    ax.bar_label(container)
plt.xticks(rotation=45);

plt.title("DenseNet121 - F1 Score (sharpening) - 1K images", fontsize=16, fontweight="bold");

In [ ]:
# vgg19_pred_sharpening = load_saved_preds("../input/nih-chest-x-ray-models/vgg19_1K_npy.npy")
# ds_121_pred_sharpening = load_saved_preds("../input/nih-chest-x-ray-models/ds121_1K_npy.npy")
# nasnet_pred_sharpening = load_saved_preds("../input/nih-chest-x-ray-models/nasnet_1K_npy.npy")
# mobile_v2_pred_sharpening = load_saved_preds("../input/nih-chest-x-ray-models/mobile_1K_npy.npy")
# effb0_pred_sharpening = load_saved_preds("../input/nih-chest-x-ray-models/effb0_1K_npy.npy")
# res50v2_pred_sharpening = load_saved_preds("../input/nih-chest-x-ray-models/resnet50_1K_npy.npy")
# inceptionv3_pred_sharpening = load_saved_preds("../input/nih-chest-x-ray-models/iv3_1K_npy.npy")

In [ ]:
BATCH_SIZE = 32
IMG_SHAPE = (224, 224)

test_datagen_aug = ImageDataGenerator(rescale=1/255., preprocessing_function=sharpening)

print("Testing images:")
test_data_aug = test_datagen_aug.flow_from_dataframe(test_data,
                                             x_col="path_gambar",
                                             y_col=labels,
                                             target_size=IMG_SHAPE,
                                             classes=labels,
                                             color_mode="rgb",
                                             class_mode="raw",
                                             shuffle=False,
                                             seed=42,
                                             batch_size=BATCH_SIZE)

In [ ]:
test_data_aug.reset()
test_x, test_y = next(test_data_aug)  
print(f"Diseases vector: {test_y[22]}")
print(f"Prediction product vector: {ds_121_pred_sharpening[22]*100}") # melihat prediksi dari model DenseNet121

In [ ]:
def plot_predictions(model_name, y_pred):
    sickest_idx = np.argsort(np.sum(test_y, axis=1) < 1)
    # 1. Setiap baris dari actual label akan dijumlahkan, apabila jumlah hasilnya 0 maka tidak ada penyakit -> "No Finding"
    # 2. Apabila terdapat penyakit / jumlah baris labelnya > 0, maka ditandakan dengan False, dan sebaliknya untuk yang No Finding dengan True
    # 3. Melakukan index sorting untuk mengurutkan dataset dari yang ada penyakit hingga No Finding

    fig, axs = plt.subplots(3, 3, figsize = (20, 20))
    fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

    for (idx, ax) in zip(sickest_idx, axs.flatten()):
        img = test_x[idx, :, :, 0] # get image
        ax.imshow(img, cmap="bone") 

        actual_label = [label[:4] for label, value in zip(labels, test_y[idx]) if value == 1] # zip prediction labels dan actual labels

        predict_label = [f'{disease[:4]}:{pred_logit*100:.2f}%' for disease, label, pred_logit in zip(labels, test_y[idx], y_pred[idx]) if (label == 1) or (pred_logit > 0.5)]

        ax.set_title(f'Actual: {", ".join(actual_label)}\nPredict: {", ".join(predict_label)}')
        plt.figtext(0.5, 0.001, 'img_size=(224, 224) | loss=binary | batch_size=32 | epochs=5 | optimizer=adam | learning_rate=0.0005 | 10 labels | *img_size_inception_v3=(299, 299)', horizontalalignment='center', fontstyle="italic")
        ax.axis('off')
    plt.suptitle(model_name, fontsize=24, fontweight="bold", x="0.5", y="1.05")

In [ ]:
plot_predictions("ResNet50V2 predictions", res50v2_pred_sharpening)

In [ ]:
plot_predictions("VGG19 predictions", vgg19_pred_sharpening)

In [ ]:
plot_predictions("DenseNet121 predictions", ds_121_pred_sharpening)

In [ ]:
plot_predictions("EfficientNetB0 predictions", effb0_pred_sharpening)

In [ ]:
plot_predictions("InceptionV3 predictions", inceptionv3_pred_sharpening)

In [ ]:
plot_predictions("NASNet-Mobile predictions", nasnet_pred_sharpening)

In [ ]:
plot_predictions("MobileNetV2 predictions", mobile_v2_pred_sharpening)

### Melihat ROC curve dari sharpening

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

def get_auc_score(pred):
    auc = roc_auc_score(test_data_aug.labels, pred)
    return auc

In [ ]:
plt.figure(figsize=(30, 20))
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

# DenseNet121
plt.subplot(331)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data_aug.labels[:, idx].astype(int), ds_121_pred_sharpening[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"DenseNet121 (avg: {get_auc_score(ds_121_pred_sharpening):.2f})", fontsize=18)

# NASNet-Mobile
plt.subplot(332)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data_aug.labels[:, idx].astype(int), nasnet_pred_sharpening[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"NASNet-Mobile (avg: {get_auc_score(nasnet_pred_sharpening):.2f})", fontsize=18)

# EffB0
plt.subplot(333)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data_aug.labels[:, idx].astype(int), effb0_pred_sharpening[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"EfficientNetB0 (avg: {get_auc_score(effb0_pred_sharpening):.2f})", fontsize=18)

# VGG19
plt.subplot(334)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data_aug.labels[:, idx].astype(int), vgg19_pred_sharpening[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"VGG19 (avg: {get_auc_score(vgg19_pred_sharpening):.2f})", fontsize=18)

# InceptionV3
plt.subplot(335)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data_aug.labels[:, idx].astype(int), inceptionv3_pred_sharpening[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"InceptionV3 (avg: {get_auc_score(inceptionv3_pred_sharpening):.2f})", fontsize=18)

# ResNet50V2
plt.subplot(336)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data_aug.labels[:, idx].astype(int), res50v2_pred_sharpening[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"ResNet50V2 (avg: {get_auc_score(res50v2_pred_sharpening):.2f})", fontsize=18)

# MobileNetV2
plt.subplot(338)
for (idx, label) in enumerate(labels):
    fpr, tpr, thresholds = roc_curve(test_data_aug.labels[:, idx].astype(int), mobile_v2_pred_sharpening[:, idx])
    plt.plot(fpr, tpr, label = f'{label} (AUC: {auc(fpr, tpr):.2f})' )
plt.legend()
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate')
plt.title(f"MobileNetV2 (avg: {get_auc_score(mobile_v2_pred_sharpening):.2f})", fontsize=18)

plt.figtext(0.5, 0.07, 'img_size=(224, 224) | loss=binary | batch_size=32 | epochs=5 | optimizer=adam | learning_rate=0.0005 | 10 labels | *img_size_inception_v3=(299, 299)', horizontalalignment='center', fontstyle="italic")
plt.suptitle("Skor AUC model pada sharpening dataset", fontsize=24, x="0.5", y="0.92", fontweight="bold")
plt.show()

In [ ]:
auc_score_sharpening_results = pd.DataFrame([get_auc_score(ds_121_pred_sharpening), get_auc_score(vgg19_pred_sharpening), get_auc_score(nasnet_pred_sharpening), get_auc_score(res50v2_pred_sharpening), get_auc_score(effb0_pred_sharpening), get_auc_score(mobile_v2_pred_sharpening), get_auc_score(inceptionv3_pred_sharpening)], columns=["auc_score"], index=["DenseNet121 + sharpening", "VGG19 + sharpening", "NASNet-Mobile + sharpening", "ResNet50V2 + sharpening", "EfficientNetB0 + sharpening", "MobileNetV2 + sharpening", "InceptionV3 + sharpening"]).sort_values(by=['auc_score'], ascending=False)
auc_score_sharpening_results.auc_score = (round(auc_score_sharpening_results.auc_score, 2)*100).astype(int)

# Bar Plot
plt.figure(figsize=(15, 7))
ax = sns.barplot(x=auc_score_sharpening_results.index, y=auc_score_sharpening_results.auc_score, color="brown");
for container in ax.containers:
    ax.bar_label(container, fontsize="11")
plt.xticks(rotation=45);
    
# plt.figtext(0.5, 0.001, 'img_size=(224, 224) | loss=binary | batch_size=32 | epochs=5 | optimizer=adam | learning_rate=0.0005 | 10 labels | *img_size_inception_v3=(299, 299)', horizontalalignment='center', fontstyle="italic", fontsize="10")
plt.title("Average AUC (sharpening) - 1K images", fontsize=15, fontweight="bold")
plt.xlabel("auc_score")
plt.ylabel("models");

In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, average_precision_score

In [ ]:
def get_pr_auc_score(test_data, preds):
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(len(labels)):
        precision[i], recall[i], _ = precision_recall_curve(test_data_aug.labels[:, i], preds[:, i])
        average_precision[i] = average_precision_score(test_data_aug.labels[:, i], preds[:, i])

    precision["macro"], recall["macro"], _ = precision_recall_curve(
        test_data_aug.labels.ravel(), preds.ravel()
    )
    average_precision["macro"] = average_precision_score(test_data_aug.labels, preds, average="macro")
    return precision, recall, average_precision

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [ ]:
def get_auc_score(pred):
    auc = roc_auc_score(test_data_aug.labels, pred)
    return auc

In [ ]:
predictions = [ds_121_pred_sharpening, vgg19_pred_sharpening, nasnet_pred_sharpening, res50v2_pred_sharpening, effb0_pred_sharpening, mobile_v2_pred_sharpening, inceptionv3_pred_sharpening]
models = ["DenseNet121", "VGG19", "NASNet-Mobile", "ResNet50V2", "EfficientNetB0", "MobileNetV2", "InceptionV3"]
linestyle = [":", "-", "--", ":", "-", "--", ":"]

_, ax = plt.subplots(figsize=(15, 10))

for i in range(len(predictions)):
    precision, recall, average_precision = get_pr_auc_score(test_data, predictions[i])
    display = PrecisionRecallDisplay(    
        recall=recall["macro"],
        precision=precision["macro"],
        average_precision=average_precision["macro"]
    )
    display.plot(ax=ax, name=f"Precision-recall for class {models[i]}", linestyle=linestyle[i])
    
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.legend(loc="best")
ax.set_title("PR-AUC Curve (sharpening) - 1K images", fontsize=16, fontweight="bold");